# Y-Flip Quantum Error Correction

In [2]:
#In case you don't have qiskit, install it now
%pip install qiskit --quiet
%pip install qiskit-aer --quiet
#Installing/upgrading pylatexenc seems to have fixed my mpl issue
#If you try this and it doesn't work, try also restarting the runtime/kernel
%pip install pylatexenc --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:

import random

#Let's go ahead and import all this stuff too
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
import numpy as np

## Encoding and Decoding

In [4]:
def encode(qc, qx):
    # Encode qubit qx[0].
    # Put your code here.

    # either of these works
    if False:
        qc.h(qx[1])
        qc.h(qx[2])
        qc.cz(qx[0],qx[1])
        qc.cz(qx[0],qx[2])
    else:
        qc.cx(qx[0], qx[1])
        qc.cx(qx[0], qx[2])
        qc.h(qx[1])
        qc.h(qx[2])

    return

In [5]:
def decode(qc, qx):
    # Decode qubit qx[0].
    # Put your code here.

    if True:
        qc.cz(qx[0],qx[2])
        qc.cz(qx[0],qx[1])
        qc.h(qx[2])
        qc.h(qx[1])
    else:
        qc.h(qx[2])
        qc.h(qx[1])
        qc.cx(qx[0], qx[1])
        qc.cx(qx[0], qx[2])

    return

## Syndrome Measurement

In [ ]:
def syndrome(qc, qx, qs):
    # Apply gates to "measure" the syndromes
    # Put your code here

    # apply Hadamards to syndrome register
    for j in range(len(qs)):
        qc.h(qs[j])

    # M0 = Z0 X1 I2
    qc.cz(qs[0], qx[0])
    qc.cx(qs[0], qx[1])

    # M1 = Z0 I1 X2
    qc.cz(qs[1], qx[0])
    qc.cx(qs[1], qx[2])

    # apply Hadamards to syndrome register
    for j in range(len(qs)):
        qc.h(qs[j])

    return

## Error Correction

In [ ]:
def qec_conditionals(qc, qx, qs, cs):
    # Correct errors using measured syndromes and conditional gates
    # Put your code here.

    # measure syndrome register
    for j in range(len(qs)):
        qc.measure(qs[j], cs[j])

    # use syndrome from measurement outcomes to apply corrections
    qc.y(qx[1]).c_if(cs,1)
    qc.y(qx[2]).c_if(cs,2)
    qc.y(qx[0]).c_if(cs,3)

    return

## Main Code Block

In [ ]:
n = 3 # number of physical qubits
k = 1 # number of logical qubits

# iterate over physical qubits
for i in range(n):

    # prepare the quantum circuit
    qx = qiskit.QuantumRegister(n)
    qs = qiskit.QuantumRegister(n-k)
    cx = qiskit.ClassicalRegister(n)
    cs = qiskit.ClassicalRegister(n-k)
    qc = qiskit.QuantumCircuit(qx, qs, cx, cs)

    # prepare a random single-qubit state in qx[0]
    theta = np.arccos(random.uniform(-1,1))
    phi   = random.uniform(0,2*np.pi)
    lamb  = 0
    qc.u3(theta, phi, lamb, qx[0])

    # encode the qubit
    encode(qc, qx)

    # apply an error to one of the encoding qubits
    qc.y(qx[i])

    # apply gates to perform a syndrome measurement
    syndrome(qc, qx, qs)

    # detect and correct error using the syndromes
    qec_conditionals(qc, qx, qs, cs)

    # decode the corrected, encoded qubit
    decode(qc, qx)

    # prepare qubit qx[0] for measurement
    qc.u3(theta, np.pi-lamb, np.pi-phi, qx[0])

    # measure the qx register
    for i in range(len(qx)):
        qc.measure(qx[i], cx[i])

    # measure the qs register
    for j in range(len(qs)):
        qc.measure(qs[j], cs[j])

    # execute the quantum circuit
    backend =  qiskit.Aer.get_backend('qasm_simulator')
    job = qiskit.execute(qc, backend, shots=1024)
    data = job.result().get_counts(qc)
    print(data)